# GDVSpectra

## Initialise GDVSpectra

### Load packages

In [1]:
%matplotlib inline
%load_ext autoreload

import os, sys
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import datacube
sys.path.append('../../../Scripts')
from dea_datahandling import load_ard
from dea_dask import create_local_dask_cluster
from dea_plotting import display_map, rgb

sys.path.append('../../modules')
import gdvspectra

sys.path.append('../../shared')
import satfetcher, tools

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Set up a dask cluster and ODC

In [2]:
# initialise the cluster. paste url into dask panel for more info.
create_local_dask_cluster()

# open up a datacube connection
dc = datacube.Datacube(app='gdvspectra')

Client Scheduler: tcp://127.0.0.1:45723 Dashboard: /user/lewis/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.11 GB


## Study area and data setup

### Set study area, time range, show map

In [3]:
# testing study area extent - yandi and roy hill
lat_extent, lon_extent = (-22.82901, -22.67901), (118.94980, 119.29979)  # yandi
#lat_extent, lon_extent = (-22.63461, -22.33461), (119.88111, 120.18111) # royhill

# display onto interacrive map
display_map(x=lon_extent, y=lat_extent)

### Load DEA ODC satellite data

In [4]:
# provide study area name
study_area = 'yandi'

# select start and end year range
time_range = ('2009', '2020')

# set datacube query parameters
platform = 'landsat'
bands = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2']
#bands = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1', 'nbart_swir_2'] # sentinel
min_gooddata = 0.90

# fetch satellite data from dea ard product
ds = satfetcher.load_dea_ard(platform=platform, 
                             bands=bands, 
                             x_extent=lon_extent, 
                             y_extent=lat_extent, 
                             time_range=time_range, 
                             min_gooddata=min_gooddata, 
                             use_dask=True)

# display dataset
#ds

Loading DEA ODC ARD satellite data.
Finding datasets
    ga_ls5t_ard_3


/home/jovyan/tenement-tools/notebooks/gdvspectra/../../../Tools/dea_tools/datahandling.py:237: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn("Setting 'min_gooddata' percentage to > 0.0 "


    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 191 out of 468 time steps with at least 90.0% good quality pixels
Applying pixel quality/cloud mask
Returning 191 time steps as a dask array
Satellite imagery fetched successfully.


### Conform band names

In [5]:
# rename dea bands to common standard
ds = satfetcher.conform_dea_ard_band_names(ds=ds, platform=platform)

# display dataset
#ds

Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.


### Make a copy of raw dataset

In [6]:
# take a copy of dataset for cva later
ds_backup = ds.copy(deep=True)

## Generate seasonal vegetation and moisture data

### Reduce to wet (jfm) and dry (son) season months

In [ ]:
# set wet and dry season month(s). we will use several per season
wet_month, dry_month = [1, 2, 3], [9, 10, 11] # note, fma and ond appear a better contrast

# get subset fo data for wet and dry season months
ds = gdvspectra.get_wet_dry_months(ds=ds, 
                                   wet_month=wet_month, 
                                   dry_month=dry_month)

# display dataset
#ds

### Calculate vegetation and moisture indices

In [ ]:
# calculate veg (mavi) and moist (ndmi) indices
ds = tools.calculate_indices(ds=ds, 
                             index=['mavi', 'ndmi'], 
                             custom_name=['veg_idx', 'mst_idx'], 
                             rescale=True, 
                             drop=True)

# display dataset
#ds

## Resample data

### Resample data to annual seasonal (djf and son) medians 

In [ ]:
# perform resampling
ds = gdvspectra.resample_to_wet_dry_medians(ds=ds, 
                                            wet_month=wet_month, 
                                            dry_month=dry_month)

# display dataset
#ds

### Persist memory

In [ ]:
# we have some calcs to make, persist now
ds = ds.persist()

## Pre-process dataset (e.g., outlier removal, interpolation)

### Remove outliers via z-score

In [ ]:
# check for outlier times and nullify whole image. warning: persists memory
ds = gdvspectra.nullify_wet_dry_outliers(ds=ds, 
                                         wet_month=wet_month, 
                                         dry_month=dry_month, 
                                         p_value=0.01)

# display dataset
#ds

### Remove any years missing wet, dry season data

In [ ]:
# drop any years from dataset where wet and dry seasons missing
ds = gdvspectra.drop_incomplete_wet_dry_years(ds)

# display dataset
#ds

### Fill empty data edges via backward and/or forward fill

In [ ]:
# ! WARNING ! this func requires pre-computed data due to xr-dask bfill and ffill bug
# xr-dask bfill/ffill will perform without error, but does not update values in ds
# todo - remove this compute when bug fixed in ver > 0.18
ds = ds.compute()

In [ ]:
# fill any empty first, last years using back/forward fill
ds = gdvspectra.fill_empty_wet_dry_edges(ds=ds,
                                         wet_month=wet_month, 
                                         dry_month=dry_month)

# display dataset
#ds

### Interpolate missing data

In [ ]:
# interpolate all missing pixels using full linear interpolation
ds = gdvspectra.interpolate_empty_wet_dry(ds=ds,
                                          method='full', 
                                          wet_month=wet_month, 
                                          dry_month=dry_month)

# display dataset
#ds

## Standardise data

### Generate invariant targets and standardise

In [ ]:
# standardise data to invariant targets derived from dry times
ds = gdvspectra.standardise_to_dry_targets(ds=ds, 
                                           dry_month=dry_month, 
                                           q_upper=0.99, 
                                           q_lower=0.05)

# display dataset
#ds

## Calculate seasonal similarity

In [ ]:
# calculate standardised seaonal similarity (diff between wet, dry per year)
ds_similarity = gdvspectra.calc_seasonal_similarity(ds=ds,
                                                    wet_month=wet_month,
                                                    dry_month=dry_month,
                                                    q_mask=0.9)

# display dataset
#ds

## Perform GDV likelihood modelling

### Generate likelihood model

In [ ]:
# generate gdv likelihood model using wet, dry, similarity variables
ds_like = gdvspectra.calc_likelihood(ds=ds, 
                                     ds_similarity=ds_similarity,
                                     wet_month=wet_month, 
                                     dry_month=dry_month)

# preview an all-time median of gdv likelihood. red is high likelihood
fig = plt.figure(figsize=(10, 5))
ds_like['like'].median('time').plot(robust=True, cmap='jet')

### Generate field occurrence points for thresholding

In [ ]:
# set location of point shapefile with presence/absence column
#shp_path = r'../GDVSDM/data_testing/presence_points/presence_points.shp'
shp_path = r'../../data/gdvspectra/yandi_2_final_albers.shp'

# read shapefile as pandas dataframe
df_records = tools.read_shapefile(shp_path=shp_path)

# subset to just x, y, pres/abse column
df_records = tools.subset_records(df_records=df_records, p_a_column='GDV_ACT')

# display dataframe
#df_records

### Threshold likelihood

In [ ]:
# perform thresholding using standard deviation on median likelihood
ds_thresh = gdvspectra.threshold_likelihood(ds=ds_like.median('time', keep_attrs=True),
                                            df=df_records, 
                                            num_stdevs=3, 
                                            res_factor=3, 
                                            if_nodata='any')

# preview an all-time median of gdv likelihood thresholded
ds_thresh.where(~ds_thresh.isnull(), 0.001)['like'].plot(robust=False, cmap='jet')

### Export GDV likelihood and threshold for later

In [ ]:
# export likelihood and thresholds   
tools.export_xr_as_nc(ds_like, filename='ds_likelihood.nc')
tools.export_xr_as_nc(ds_thresh, filename='ds_threshold.nc')

## Perform trend analysis

### Calculate trends using Mann-Kendall trend analysis

In [ ]:
# create a mask where gdv highly likely
ds_mask = xr.where(~ds_thresh.isnull(), True, False)

# do mk to find sig. inc/dec trends in high likelihood areas
ds_mk = gdvspectra.perform_mk_original(ds=ds_like.where(ds_mask), 
                                        pvalue=None, 
                                        direction='both')

# show mk trends. blue is increasing, red is decreasing
ds_mk['tau'].plot(robust=True, cmap='Spectral')

### Calculate slope using Theil-Sen

In [ ]:
# create a mask where gdv highly likely
ds_mask = xr.where(~ds_thresh.isnull(), True, False)

# do theil sen slopes in high likelihood areas
ds_ts = gdvspectra.perform_theilsen_slope(ds=ds_like.where(ds_mask), 
                                          alpha=0.95)

# show mk trends. blue is increasing, red is decreasing
ds_ts['theilsen'].plot(robust=True, cmap='Spectral')

## Peform Change Vector Analysis (CVA)

### Get specific months from dataset copy

In [7]:
# get months we want to assess across years during trend analysis
ds_backup = gdvspectra.get_trend_months(ds_backup, trend_month=[9, 10, 11])

# display dataset
#ds_backup

Getting requested months for trend analysis.
Reducing dataset to months ([9, 10, 11]) for trend analysis.
Got trend months successfully.


### Generate tasselled cap greenness and brightness

In [8]:
# calculate tasselled cap greenness and brightness indices
ds_backup = tools.calculate_indices(ds=ds_backup, 
                                    index=['tcg', 'tcb'],
                                    rescale=False, 
                                    drop=True)

# display dataset
#ds_backup

Calculating indices.
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.


### Resample annual medians for selected dates

In [9]:
# reduce all selected months into annual medians
ds_backup = gdvspectra.resample_to_annual_medians(ds_backup)

Resampling dataset down to annual medians.
Resampled down to annual medians successfully.


### Interpolate missing data

In [10]:
# interpolate all missing pixels using full linear interpolation
ds_backup = gdvspectra.interpolate_empty(ds=ds_backup,
                                         method='full')

# display dataset
#ds_backup

Interpolating empty values in dataset.
Interpolating using full method. This can take awhile. Please wait.
Interpolated empty values successfully.


### Standardise using invariant targets

In [16]:
# standardise to targets
ds_backup = gdvspectra.standardise_to_targets(ds_backup, 
                                              q_upper=0.99, 
                                              q_lower=0.05)

Standardising data using invariant targets.
Generating invariant targets.
Got orthogonal polynomial coefficients: [-11.  -9.  -7.  -5.  -3.  -1.   1.   3.   5.   7.   9.  11.]
Got sum of squares: 572 and constant 2: 
Standardising to invariant targets and rescaling via increasing sigmoidal.
Standardised using invariant targets successfully.


### Perform Change Vector Analysis

In [ ]:
# create a baseline and comparison dataset split
# can be one baseline image, a set of baselines, or all time
# same with comparison dates, one, many, or alltime

In [ ]:
# create a empty dataset via full_like with nan
# add bunch of vars to it like angout, magout, magnitude etc

In [ ]:
# persist
ds_backup = ds_backup.persist()

In [ ]:
# generate cva comparing latest three years to baseline of 1999-2010
ds_cva = gdvspectra.perform_cva(ds=ds_backup.compute(),
                                base_times=(1999, 2010),
                                comp_times=(2018, 2020),
                                reduce_comp=True,
                                vege_var = 'tcg',
                                soil_var = 'tcb',
                                tmf=2)

# plot magnitude of all change from last 3 years to baseline decade
ds_cva['magnitude'].isel(time=0).plot(robust=True, cmap='Reds')

### Isolate specific change type

In [ ]:
# isolate angles between 90-180 degrees to focus on veg decline only
ds_dec = isolate_cva_change(ds_cva, angle_min=90, angle_max=180)

# plot magnitude for veg decline areas
ds_dec['magnitude'].plot(robust=True, cmap='Reds')

### Intersect veg decline with GDV likelihood areas

In [ ]:
# create a mask where gdv highly likely
ds_mask = xr.where(~ds_thresh.isnull(), True, False)

# select change areas where gdv exists
ds_final

